# T6.2 Redes Bayesianas

# Índice

1. Ejemplos
2. Un ejemplo detallado: aspersor
3. Otro ejemplo: cáncer
4. Redes bayesianas
5. Algunas redes bayesianas simples
6. Otro ejemplo de red bayesiana: modelo oculto de Markov

## 1 Ejemplos

**Factorización de una distribución conjunta con dependencias inexistentes o despreciables:** $\;$ la factorización exacta (o aproximada) puede ser incompleta, lo que queda reflejado en el grafo correspondiente

**Ejemplo:** $\;P(a,b,c)=P(a)~P(b)~P(c \mid a,b)$

In [1]:
import graphviz; graphviz.Source('digraph { rankdir=LR; a -> c b -> c}').render(filename='ejemplo1', format='svg');

<div align=center><img src="ejemplo1.svg"/></div>

**Ejemplo:** $\;P(a,b,c,d,e,f)=P(a)~P(b)~P(c)~P(d\mid b,c)~P(e\mid a,b,d)~P(f\mid a,c)$

In [2]:
import graphviz; graphviz.Source('''digraph { rankdir=LR; a -> {e, f} b -> {d, e} c -> {d, f} d -> e}
''').render(filename='ejemplo2', format='svg');


<div align=center><img src="ejemplo2.svg"/></div>


## 2 Un ejemplo detallado: aspersor

In [3]:
import graphviz; graphviz.Source('''digraph { rankdir=LR; node [shape=plaintext];
Lluvia [label=<<TABLE BORDER="0" CELLBORDER="1" CELLSPACING="0">
<TR><TD COLSPAN="2">P(L)</TD></TR>
<TR><TD COLSPAN="2">Lluvia</TD></TR>
<TR><TD>no llueve</TD><TD>sí llueve</TD></TR>
<TR><TD>0.8</TD><TD>0.2</TD></TR>
</TABLE>>];
Aspersor [label=<<TABLE BORDER="0" CELLBORDER="1" CELLSPACING="0">
<TR><TD COLSPAN="3">P(A | L)</TD></TR>
<TR><TD></TD><TD COLSPAN="2">Aspersor</TD></TR>
<TR><TD>Lluvia</TD><TD>parado</TD><TD>funciona</TD></TR>
<TR><TD>no llueve</TD><TD>0.60</TD><TD>0.40</TD></TR>
<TR><TD>sí llueve</TD><TD>0.99</TD><TD>0.01</TD></TR>
</TABLE>>];
Césped [label=<<TABLE BORDER="0" CELLBORDER="1" CELLSPACING="0">
<TR><TD COLSPAN="4">P(C | L, A)</TD></TR>
<TR><TD COLSPAN="2"></TD><TD COLSPAN="2">Césped</TD></TR>
<TR><TD>Lluvia</TD><TD>Aspersor</TD><TD>reseco</TD><TD>mojado</TD></TR>
<TR><TD>no llueve</TD><TD>parado</TD><TD>1.00</TD><TD>0.00</TD></TR>
<TR><TD>no llueve</TD><TD>funciona</TD><TD>0.10</TD><TD>0.90</TD></TR>
<TR><TD>sí llueve</TD><TD>parado</TD><TD>0.20</TD><TD>0.80</TD></TR>
<TR><TD>sí llueve</TD><TD>funciona</TD><TD>0.01</TD><TD>0.99</TD></TR>
</TABLE>>];
Lluvia -> Aspersor -> Césped
Lluvia -> Césped
}''').render(filename='aspersor', format='svg');

<div align="center"><img src='aspersor.svg'/></div>

**Distribución conjunta:** $\quad P(L,A,C)=P(L)P(A\mid L)P(C\mid L, A)$

**Ejercicio:** $\;$ calcular $P(L=l, A=a, C=c)\;$ para todo $\,l\in\{\mathrm{n,s}\},\,$ $a\in\{\mathrm{p,f}\}\,$ y $\,c\in\{\mathrm{r,m}\}$

**Solución:**
$$\begin{array}{cccc}
L & A & C & P(L,A,C)\\\hline
\mathrm{n} & \mathrm{p} & \mathrm{r} & 0.8\cdot 0.60\cdot 1.00 = 0.48000\\
\mathrm{n} & \mathrm{p} & \mathrm{m} & 0.8\cdot 0.60\cdot 0.00 = 0.00000\\
\mathrm{n} & \mathrm{f} & \mathrm{r} & 0.8\cdot 0.40\cdot 0.10 = 0.03200\\
\mathrm{n} & \mathrm{f} & \mathrm{m} & 0.8\cdot 0.40\cdot 0.90 = 0.28800\\
\mathrm{s} & \mathrm{p} & \mathrm{r} & 0.2\cdot 0.99\cdot 0.20 = 0.03960\\
\mathrm{s} & \mathrm{p} & \mathrm{m} & 0.2\cdot 0.99\cdot 0.80 = 0.15840\\
\mathrm{s} & \mathrm{f} & \mathrm{r} & 0.2\cdot 0.01\cdot 0.01 = 0.00002\\
\mathrm{s} & \mathrm{f} & \mathrm{m} & 0.2\cdot 0.01\cdot 0.99 = 0.00198
\end{array}$$

**Ejercicio:** $\;$ ¿Cuál es la probabilidad de que llueva si el césped está mojado?

**Solución:**
$$\begin{align*}
P(L=\mathrm{s} \mid C=\mathrm{m})&=\frac{P(L=\mathrm{s},C=\mathrm{m})}{P(C=\mathrm{m})}\\
&=\frac{\sum_aP(L=\mathrm{s},A=a,C=\mathrm{m})}{\sum_{l,\,a}P(L=l,A=a,C=\mathrm{m})}\\
&=\frac{0.1584+0.00198}{0.0+0.288+0.1584+0.00198}=0.3577
\end{align*}$$


**Ejercicio:** $\;$ El césped está mojado. ¿Cuál es la mejor predicción: llueve o no llueve?

**Solución:** $\;\operatorname{argmax}_{l\in\{\mathrm{n,s}\}} P(L=l \mid C=\mathrm{m}) ~=~ \mathrm{n}$

**Ejercicio:** $\;$ calcular $P(A=a\mid L=l, C=c),\,a\in\{\mathrm{p,f}\},\,l\in\{\mathrm{n,s}\},\,c\in\{\mathrm{r,m}\}$

**Solución:** $\displaystyle\;P(A=a\mid L=l, C=c)=\frac{P(L=l, A=a, C=c)}{P(L=l, C=c)}=\frac{P(L=l, A=a, C=c)}{\sum_{\tilde{a}}P(L=l, A=\tilde{a}, C=c)}$
$$\begin{array}{cccccc}
L & A & C & P(L,A,C) & P(L,C) & P(A\mid L, C)\\\hline
\mathrm{n} & \mathrm{p} & \mathrm{r} & 0.48000 & 0.51200 & 0.93750\\
\mathrm{n} & \mathrm{p} & \mathrm{m} & 0.00000 & 0.28800 & 0.00000\\
\mathrm{n} & \mathrm{f} & \mathrm{r} & 0.03200 & 0.51200 & 0.06250\\
\mathrm{n} & \mathrm{f} & \mathrm{m} & 0.28800 & 0.28800 & 1.00000\\
\mathrm{s} & \mathrm{p} & \mathrm{r} & 0.03960 & 0.03962 & 0.99950\\
\mathrm{s} & \mathrm{p} & \mathrm{m} & 0.15840 & 0.16038 & 0.98765\\
\mathrm{s} & \mathrm{f} & \mathrm{r} & 0.00002 & 0.03962 & 0.00050\\
\mathrm{s} & \mathrm{f} & \mathrm{m} & 0.00198 & 0.16038 & 0.01235
\end{array}$$

**Ejercicio:** $\;$ Llueve y el cesped está mojado. ¿Cuál es la mejor predicción sobre el estado del aspersor?

**Solución:** $\;\operatorname{argmax}_{a\in\{\mathrm{p,f}\}} P(A=a \mid L=\mathrm{s}, C=\mathrm{m}) ~=~ \mathrm{p}\quad$ con probabilidad del $\;98.8\%$

# 3 Otro ejemplo: cáncer

In [4]:
import graphviz; graphviz.Source('''digraph { rankdir=LR; node [shape=plaintext];
Polución [label=<<TABLE BORDER="0" CELLBORDER="1" CELLSPACING="0">
<TR><TD COLSPAN="2">P(P)</TD></TR>
<TR><TD COLSPAN="2">Polución</TD></TR>
<TR><TD>bajo</TD><TD>alto</TD></TR>
<TR><TD>0.9</TD><TD>0.1</TD></TR>
</TABLE>>];
Fumador [label=<<TABLE BORDER="0" CELLBORDER="1" CELLSPACING="0">
<TR><TD COLSPAN="2">P(F)</TD></TR>
<TR><TD COLSPAN="2">Fumador</TD></TR>
<TR><TD>no</TD><TD>sí</TD></TR>
<TR><TD>0.7</TD><TD>0.3</TD></TR>
</TABLE>>];
Cáncer [label=<<TABLE BORDER="0" CELLBORDER="1" CELLSPACING="0">
<TR><TD COLSPAN="4">P(C | P, F)</TD></TR>
<TR><TD COLSPAN="2"></TD><TD COLSPAN="2">Cáncer</TD></TR>
<TR><TD>Polución</TD><TD>Fumador</TD><TD>negativo</TD><TD>positivo</TD></TR>
<TR><TD>bajo</TD><TD>no</TD><TD>0.999</TD><TD>0.001</TD></TR>
<TR><TD>bajo</TD><TD>sí</TD><TD>0.970</TD><TD>0.030</TD></TR>
<TR><TD>alto</TD><TD>no</TD><TD>0.950</TD><TD>0.050</TD></TR>
<TR><TD>alto</TD><TD>sí</TD><TD>0.920</TD><TD>0.080</TD></TR>
</TABLE>>];
RayosX [label=<<TABLE BORDER="0" CELLBORDER="1" CELLSPACING="0">
<TR><TD COLSPAN="4">P(X | C)</TD></TR>
<TR><TD></TD><TD COLSPAN="3">Rayos X</TD></TR>
<TR><TD>Cáncer</TD><TD>negativo</TD><TD>dudoso</TD><TD>positivo</TD></TR>
<TR><TD>negativo</TD><TD>0.80</TD><TD>0.10</TD><TD>0.10</TD></TR>
<TR><TD>positivo</TD><TD>0.10</TD><TD>0.20</TD><TD>0.70</TD></TR>
</TABLE>>];
Disnea [label=<<TABLE BORDER="0" CELLBORDER="1" CELLSPACING="0">
<TR><TD COLSPAN="3">P(D | C)</TD></TR>
<TR><TD></TD><TD COLSPAN="2">Disnea</TD></TR>
<TR><TD>Cáncer</TD><TD>no</TD><TD>sí</TD></TR>
<TR><TD>negativo</TD><TD>0.70</TD><TD>0.30</TD></TR>
<TR><TD>positivo</TD><TD>0.35</TD><TD>0.65</TD></TR>
</TABLE>>];
{ Polución, Fumador } -> Cáncer -> {RayosX, Disnea}
}''').render(filename='cancer', format='svg');

<div align="center"><img src='cancer.svg'/></div>

**Ejercicio:** ¿Cuál es la probabilidad de que un paciente no fumador no tenga cáncer si la radiografía ha dado un resultado negativo pero sufre de disnea?

**Solución:** $\;P(P,F,C,X,D)=P(P)P(F)P(C\mid P,F)P(X\mid C)P(D\mid C)$
$$\begin{align*}
P(C=n\mid F=n, X=n, D=s)&=\frac{P(F=n, C=n, X=n, D=s)}{P(F=n, X=n, D=s)}\\
&=\frac{\sum_p P(P=p, F=n, C=n, X=n, D=s)}{\sum_{p,c} P(P=p, F=n, C=c, X=n, D=s)}\\
&=\frac{0.15105+0.01596}{0.15105+0.00004+0.01596+0.00023}=99.8\%
\end{align*}$$

$$\begin{array}{ccccccc}
P & F & C & X & D & P(P, F, C, X, D)\\\hline
\mathrm{b}&\mathrm{n}&\mathrm{n}&\mathrm{n}&\mathrm{s}&0.9\cdot0.7\cdot0.999\cdot0.80\cdot0.30=0.15105\\
\mathrm{b}&\mathrm{n}&\mathrm{p}&\mathrm{n}&\mathrm{s}&0.9\cdot0.7\cdot0.001\cdot0.10\cdot0.65=0.00004\\
\mathrm{a}&\mathrm{n}&\mathrm{n}&\mathrm{n}&\mathrm{s}&0.1\cdot0.7\cdot0.950\cdot0.80\cdot0.30=0.01596\\
\mathrm{a}&\mathrm{n}&\mathrm{p}&\mathrm{n}&\mathrm{s}&0.1\cdot0.7\cdot0.050\cdot0.10\cdot0.65=0.00023
\end{array}$$

## 4 Redes bayesianas

**Red bayesiana:** $\;$ grafo dirigido y acíclico (DAG)

**Nodos:** $\;$ cada nodo se asocia a una variable aleatoria discreta (o continua), $\,x_i,\,$ cuya distribución de probabilidad se halla condicionada por los valores observados de las variables asociadas a sus nodos padre o antepasado $\;a(x_i)$

**Arcos:** $\;$ representan dependencias entre variables; cada ausencia de arista indica una independencia condicional

**Distribución conjunta:** $\;$ de una red con nodos $\;x_1,\dotsc,x_D$
$$P(x_1,\dotsc,x_D)=\prod_{i=1}^DP(x_i\mid a(x_i))$$

## 5 Algunas redes bayesianas simples

**Tarea:** $\;$ clasificar un $\,\boldsymbol{x}=(x_1,\dotsc,x_D)^t\in\mathbb{R}^D\,$ dado en una clase $\,c\in\{1,\dotsc,C\}$

### Red general

In [5]:
import graphviz; graphviz.Source('digraph {rankdir=TB; x[label=<<B><I>x</I></B>>] c->x}').render(filename='bayes', format='svg');

<div><table border-collapse: collapse align=left><tr>
<td style="border: none; text-align:left; vertical-align:top; padding:0; margin:0;" width=600>

**Red:** $\;c\,$ depende de $\,\boldsymbol{x}\,$ sin más (asunciones)

**Distribución conjunta:** $\displaystyle\;P(\boldsymbol{x}, c)=P(c)P(\boldsymbol{x}\mid c)$

**Clasificador de Bayes:** $\displaystyle\;P(c\mid\boldsymbol{x})=\frac{P(\boldsymbol{x},c)}{P(\boldsymbol{x})}=\frac{P(c)P(\boldsymbol{x}\mid c)}{\sum_{\tilde{c}}P(\tilde{c})P(\boldsymbol{x}\mid\tilde{c})}$

</td><td style="border: none; text-align:left; vertical-align:top; padding:0; margin:0;" width=200>

<img src="bayes.svg">

</td></tr></table></div>

### Red naive Bayes

In [1]:
import graphviz; graphviz.Source('''digraph {rankdir=TB; splines=false;
{rank=same x1[label=<<i>x<sub>1</sub></i>>] xdots[shape=none label="&hellip;"] xD[label=<<i>x<sub>D</sub></i>>]}
c->{x1,xD} c->xdots[style=invis]}''').render(filename='naive', format='svg');

<div><table border-collapse: collapse align=left><tr>
<td style="border: none; text-align:left; vertical-align:top; padding:0; margin:0;" width=600>

**Red:** $\;c\,$ depende de $\,x_1,\dotsc,x_D,\,$ y las características son condicionalmente independientes dada la clase
$$P(x_1,\dotsc,x_D\mid c)=\prod_{d=1}^DP(x_d\mid c)$$

**Distribución conjunta:** $\displaystyle\;P(x_1,\dotsc,x_D,c)=P(c)\prod_{d=1}^DP(x_d\mid c)$

**Clasificador naive Bayes:**
$$P(c\mid x_1,\dotsc,x_D)=\frac{P(x_1,\dotsc,x_D,c)}{P(x_1,\dotsc,x_D)}=\frac{P(c)\prod_{d=1}^DP(x_d\mid c)}{\sum_{\tilde{c}}P(\tilde{c})\prod_{d=1}^DP(x_d\mid\tilde{c})}$$

</td><td style="border: none; text-align:left; vertical-align:top; padding:0; margin:0;" width=300>

<img src="naive.svg">

</td></tr></table></div>

## 6 Otro ejemplo de red bayesiana: modelo oculto de Markov